# Verificación de Consistencia entre Columnas 'specie' y 'directory'

Este notebook está diseñado para detectar y corregir inconsistencias entre las columnas 'specie' y 'directory' en el dataset de vocalizaciones de primates. En particular, verificamos que el código de especie en la columna 'specie' coincida con las últimas dos letras del código en la columna 'directory' (después de '__').

In [1]:
# Importamos las bibliotecas necesarias
import os
import pandas as pd
import re
import sys
import matplotlib.pyplot as plt
from IPython.display import display

# Agregamos la carpeta raíz del proyecto al path para poder importar módulos propios
PROJECT_ROOT = os.path.join(os.getcwd(), "..")
sys.path.append(PROJECT_ROOT)

# Definimos las rutas de los directorios
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
PROCESSED_DATA_SUBDIR = os.path.join(DATA_DIR, "processed")
PROCESSED_DATASET_FILE = os.path.join(PROCESSED_DATA_SUBDIR, "processed_dataset.csv")

print(f"Proyecto: {PROJECT_ROOT}")
print(f"Archivo de datos: {PROCESSED_DATASET_FILE}")

Proyecto: /home/nhrot/Programming/Python/banana-net/notebooks/..
Archivo de datos: /home/nhrot/Programming/Python/banana-net/notebooks/../data/processed/processed_dataset.csv


In [2]:
# Cargamos el dataset procesado
try:
    df = pd.read_csv(PROCESSED_DATASET_FILE)
    print(f"Dataset cargado exitosamente con {len(df)} filas.")
    print("\nPrimeras 5 filas:")
    display(df.head())
    
    print("\nColumnas disponibles:")
    print(df.columns.tolist())
    
    # Verificamos que existan las columnas necesarias
    required_columns = ['directory', 'specie']
    missing_columns = [col for col in required_columns if col not in df.columns]
    
    if missing_columns:
        print(f"\n⚠️ Faltan las siguientes columnas: {missing_columns}")
    else:
        print("\n✓ Todas las columnas requeridas están presentes.")
        
except Exception as e:
    print(f"Error al cargar el dataset: {e}")

Dataset cargado exitosamente con 16585 filas.

Primeras 5 filas:


,begin_time,end_time,low_freq,high_freq,inband_power,specie,call_type,recording_file,directory
0,0.698818,0.772731,0.000,3739.187,-70.48,aa,gc,20240117_051057.wav,night_monkey__AA
1,1.211587,1.282141,70.551,4338.868,-72.05,aa,gc,20240117_051057.wav,night_monkey__AA
2,1.727301,1.796175,70.551,4338.868,-68.17,aa,gc,20240117_051057.wav,night_monkey__AA
3,3.330185,3.395699,105.826,4444.694,-71.47,aa,gc,20240117_051057.wav,night_monkey__AA
4,3.861571,3.940524,176.377,4338.868,-73.33,aa,gc,20240117_051057.wav,night_monkey__AA



Columnas disponibles:
['begin_time', 'end_time', 'low_freq', 'high_freq', 'inband_power', 'specie', 'call_type', 'recording_file', 'directory']

✓ Todas las columnas requeridas están presentes.


## Funciones para Verificación de Consistencia

Definimos dos funciones:
1. Una para detectar inconsistencias entre las columnas 'specie' y 'directory'
2. Otra para corregir estas inconsistencias

In [3]:
from src.banana_net.utils.preprocessing import check_species_directory_consistency, fix_species_directory_consistency

## Aplicación de las Funciones de Verificación y Corrección

In [4]:
# Aplicamos la función para verificar inconsistencias
inconsistency_results = check_species_directory_consistency(df)

# Mostramos el número total de inconsistencias
print(f"Se encontraron {inconsistency_results['count']} inconsistencias entre 'specie' y 'directory'")

# Mostramos el resumen por directorio
print("\n===== Resumen de Inconsistencias por Directorio =====")
display(inconsistency_results['summary'])

# Mostramos algunos ejemplos de inconsistencias
if inconsistency_results['count'] > 0:
    print(f"\n===== Ejemplos de Inconsistencias (mostrando hasta 10) =====")
    display(inconsistency_results['inconsistencies'].head(10))

Se encontraron 55 inconsistencias entre 'specie' y 'directory'

===== Resumen de Inconsistencias por Directorio =====


,directory,specie,expected_specie,count
0,howler_monkey__AS,ac,as,14
1,large_headed_capuchin__SM,lw,sm,1
2,peruvian_spider_monkey__AC,as,ac,34
3,peruvian_spider_monkey__AC,sm,ac,6



===== Ejemplos de Inconsistencias (mostrando hasta 10) =====


,begin_time,end_time,low_freq,high_freq,inband_power,specie,call_type,recording_file,directory,expected_specie
2643,5.676740,6.369719,110.884,918.750,-40.03,ac,bc,20240823_142122.wav,howler_monkey__AS,as
2644,7.124572,7.594808,95.043,871.228,-40.26,ac,bc,20240823_142122.wav,howler_monkey__AS,as
2645,8.324911,8.807521,95.043,887.069,-37.16,ac,bc,20240823_142122.wav,howler_monkey__AS,as
2646,9.809867,10.205855,142.565,696.983,-48.89,ac,bc,20240823_142122.wav,howler_monkey__AS,as
2647,11.665223,11.986963,95.043,871.228,-33.05,ac,bc,20240823_142122.wav,howler_monkey__AS,as
2648,14.103025,14.449515,79.203,855.388,-32.77,ac,bc,20240823_142122.wav,howler_monkey__AS,as
2649,15.459970,15.769336,95.043,744.504,-40.47,ac,bc,20240823_142122.wav,howler_monkey__AS,as
2650,16.759306,17.105796,95.043,728.664,-50.07,ac,bc,20240823_142122.wav,howler_monkey__AS,as
2651,18.330884,18.627875,110.884,744.504,-40.98,ac,bc,20240823_142122.wav,howler_monkey__AS,as
2652,20.469598,20.778964,126.724,728.664,-41.34,ac,bc,20240823_142122.wav,howler_monkey__AS,as


In [5]:
# Si hay inconsistencias, aplicamos la función de corrección
if inconsistency_results['count'] > 0:
    # Guardamos el número original de filas
    original_count = len(df)
    
    # Aplicamos la función para corregir inconsistencias
    corrected_df = fix_species_directory_consistency(df)
    
    # Verificamos que todas las inconsistencias hayan sido corregidas
    post_check = check_species_directory_consistency(corrected_df)
    
    print(f"\n===== Resultados de la Corrección =====")
    print(f"Filas antes de la corrección: {original_count}")
    print(f"Filas después de la corrección: {len(corrected_df)}")
    print(f"Inconsistencias restantes: {post_check['count']}")
    
    # Mostramos las primeras filas del DataFrame corregido
    print("\nPrimeras 5 filas del DataFrame corregido:")
    display(corrected_df.head())
    
    # Opción para guardar el DataFrame corregido
    save_path = os.path.join(PROCESSED_DATA_SUBDIR, "corrected_dataset.csv")
    print(f"\n¿Deseas guardar el dataset corregido en {save_path}?")
    print("Para guardar, ejecuta la siguiente celda.")
else:
    print("\nNo hay inconsistencias para corregir. El dataset está limpio.")


===== Resultados de la Corrección =====
Filas antes de la corrección: 16585
Filas después de la corrección: 16585
Inconsistencias restantes: 0

Primeras 5 filas del DataFrame corregido:


,begin_time,end_time,low_freq,high_freq,inband_power,specie,call_type,recording_file,directory
0,0.698818,0.772731,0.000,3739.187,-70.48,aa,gc,20240117_051057.wav,night_monkey__AA
1,1.211587,1.282141,70.551,4338.868,-72.05,aa,gc,20240117_051057.wav,night_monkey__AA
2,1.727301,1.796175,70.551,4338.868,-68.17,aa,gc,20240117_051057.wav,night_monkey__AA
3,3.330185,3.395699,105.826,4444.694,-71.47,aa,gc,20240117_051057.wav,night_monkey__AA
4,3.861571,3.940524,176.377,4338.868,-73.33,aa,gc,20240117_051057.wav,night_monkey__AA



¿Deseas guardar el dataset corregido en /home/nhrot/Programming/Python/banana-net/notebooks/../data/processed/corrected_dataset.csv?
Para guardar, ejecuta la siguiente celda.


In [ ]:
# Descomentar y ejecutar para guardar el dataset corregido
corrected_df.to_csv(os.path.join(PROCESSED_DATA_SUBDIR, "corrected_dataset.csv"), index=False)
print(f"Dataset corregido guardado en {os.path.join(PROCESSED_DATA_SUBDIR, 'corrected_dataset.csv')}")